# Finding Dory
## Politecnico di Milano, Internet of Things + Wireless Internet project
### Processing and localization

In the fist part of this project we have collected from the CoAP and MQTT server a total of 180 fingerprints and Dory's RSSI. In this part of the project we analyze the data and localize Dory using fingerprint-based methods.


# Import data

The fist step is importing the retrieved data and set it up in a format suitable for analysis. In this section we import the data by parsing with simple custom parsers the hand-crafted text files and convert the entire dataset into a standard dataframe, suitable for further analysis.

In [2]:
import ast
import pandas as pd
import numpy as np
from numpy.ma.core import log10

## Import data from CoAP

In [ ]:
data_coap = []
with open("coap/coap_coords.txt") as f:
  for l in f:
    row = []
    ls = l.split("||")

    # extract ground truth position
    pos = tuple(map(int, map(float,ls[0].split(","))))

    # parse list and integet literals as python lists
    anchors_readings = []
    for anchor_readings in ls[1:]:
      anchors_readings.append(list(map(ast.literal_eval , ast.literal_eval(anchor_readings.strip()))))
    
    # unroll experiments into one row per experiment
    # appending ground truth and one station per column
    for exp_idx in range(len(anchors_readings[0])):
      row = [pos]
      for anchor_idx in range(len(anchors_readings)):
        #print(anchors_readings[anchor_idx])
        row.append(anchors_readings[anchor_idx][exp_idx])
      data_coap.append(row)
  
  # build df
  df_coap = pd.DataFrame(data=data_coap, columns=['gt_x_y', "A1", "A2", "A3", "A4", "A5", "A6"])
  display(df_coap)

,gt_x_y,A1,A2,A3,A4,A5,A6
0,"(10, 6)",-64,-64,-61,-59,-59,-55
1,"(10, 6)",-60,-60,-57,-55,-55,-51
2,"(10, 6)",-62,-61,-58,-56,-57,-53
3,"(10, 6)",-66,-66,-63,-61,-61,-57
4,"(10, 6)",-64,-64,-61,-59,-59,-55
...,...,...,...,...,...,...,...
105,"(8, 2)",-62,-64,-55,-52,-62,-62
106,"(8, 2)",-68,-70,-61,-58,-68,-69
107,"(8, 2)",-64,-67,-57,-55,-64,-64
108,"(8, 2)",-70,-72,-63,-60,-70,-70


## Import data from MQTT

In [ ]:
data_mqtt = []
with open("mqtt/mqtt_coords.txt") as f:
  for l in f:
    row = []
    ls = l.split("|")

    # extract ground truth position
    pos = tuple(map(int, map(float,ls[0].split(","))))

    # parse list and integet literals as python lists
    anchors_readings = []
    for anchor_readings in ls[1:]:
      anchors_readings.append(list(map(int , anchor_readings.strip().split(","))))
    
    # unroll experiments into one row per experiment
    # appending ground truth and one station per column
    for exp_idx in range(len(anchors_readings[0])):
      row = [pos]
      for anchor_idx in range(len(anchors_readings)):
        #print(anchors_readings[anchor_idx])
        row.append(anchors_readings[anchor_idx][exp_idx])
      data_mqtt.append(row)
  
  # build df
  df_mqtt = pd.DataFrame(data=data_mqtt, columns=['gt_x_y', "A1", "A2", "A3", "A4", "A5", "A6"])
  display(df_mqtt)

,gt_x_y,A1,A2,A3,A4,A5,A6
0,"(10, 2)",-63,-65,-58,-49,-62,-61
1,"(10, 2)",-61,-63,-56,-48,-65,-59
2,"(10, 2)",-63,-65,-57,-49,-63,-61
3,"(10, 2)",-69,-71,-63,-55,-68,-67
4,"(10, 2)",-63,-65,-58,-50,-63,-61
...,...,...,...,...,...,...,...
65,"(8, 4)",-59,-61,-54,-55,-57,-56
66,"(8, 4)",-66,-67,-61,-60,-63,-63
67,"(8, 4)",-64,-65,-59,-58,-61,-61
68,"(8, 4)",-65,-66,-60,-59,-62,-62


## Merge all fingerprints in a unique df
Also set Dory's RSSI

In [ ]:
df = pd.concat([df_coap, df_mqtt])

Dory_RSSI = [-57,-63,-58,-64,-63,-66]
Dory_RSSI_np = np.array(Dory_RSSI)
Dory_RSSI_mw = np.power(10, Dory_RSSI_np/10)

# verify that all data is present
assert(len(df) == 6*6*5)
assert(df.isna().sum().sum() == 0)
display(df)

,gt_x_y,A1,A2,A3,A4,A5,A6
0,"(10, 6)",-64,-64,-61,-59,-59,-55
1,"(10, 6)",-60,-60,-57,-55,-55,-51
2,"(10, 6)",-62,-61,-58,-56,-57,-53
3,"(10, 6)",-66,-66,-63,-61,-61,-57
4,"(10, 6)",-64,-64,-61,-59,-59,-55
...,...,...,...,...,...,...,...
65,"(8, 4)",-59,-61,-54,-55,-57,-56
66,"(8, 4)",-66,-67,-61,-60,-63,-63
67,"(8, 4)",-64,-65,-59,-58,-61,-61
68,"(8, 4)",-65,-66,-60,-59,-62,-62


In [ ]:
np.array(log10(Dory_RSSI_mw)*10)

array([-57., -63., -58., -64., -63., -66.])

### Naive minimum euclidean distance

Compute euclidean distance of each figerprint from the sample

In [ ]:
def compute_euclidean_distance(row):
  return np.linalg.norm(Dory_RSSI - row[["A1", "A2", "A3", "A4", "A5", "A6"]])

def compute_euclidean_distance_mw(row):
  return np.linalg.norm(Dory_RSSI_mw - np.power(10, row[["A1", "A2", "A3", "A4", "A5", "A6"]] / 10))

df['euclidean_distance'] = df.apply(compute_euclidean_distance, axis=1)
df['euclidean_distance_mw'] = df.apply(compute_euclidean_distance_mw, axis=1)
display(df)

,gt_x_y,A1,A2,A3,A4,A5,A6,euclidean_distance,euclidean_distance_mw
0,"(10, 6)",-64,-64,-61,-59,-59,-55,14.866069,0.000004
1,"(10, 6)",-60,-60,-57,-55,-55,-51,19.723083,0.000009
2,"(10, 6)",-62,-61,-58,-56,-57,-53,17.262677,0.000006
3,"(10, 6)",-66,-66,-63,-61,-61,-57,14.456832,0.000003
4,"(10, 6)",-64,-64,-61,-59,-59,-55,14.866069,0.000004
...,...,...,...,...,...,...,...,...,...
65,"(8, 4)",-59,-61,-54,-55,-57,-56,15.524175,0.000005
66,"(8, 4)",-66,-67,-61,-60,-63,-63,11.445523,0.000002
67,"(8, 4)",-64,-65,-59,-58,-61,-61,10.908712,0.000002
68,"(8, 4)",-65,-66,-60,-59,-62,-62,10.908712,0.000002


### Show the smallest distances from fingerprint for visual inspection

In [ ]:
display(df.nsmallest(20,'euclidean_distance', 'all'))

,gt_x_y,A1,A2,A3,A4,A5,A6,euclidean_distance,euclidean_distance_mw
48,"(2, 4)",-58,-61,-59,-64,-62,-65,2.828427,6.177862e-07
47,"(2, 4)",-58,-61,-59,-64,-61,-65,3.316625,6.713801e-07
85,"(2, 2)",-54,-63,-56,-63,-64,-66,3.872983,2.196362e-06
53,"(0, 4)",-57,-61,-61,-66,-63,-66,4.123106,8.558678e-07
87,"(2, 2)",-54,-61,-56,-63,-63,-65,4.358899,2.214394e-06
21,"(4, 4)",-61,-62,-57,-62,-61,-64,5.477226,1.337618e-06
54,"(0, 4)",-58,-62,-62,-67,-64,-68,5.656854,1.074193e-06
89,"(2, 2)",-53,-62,-55,-62,-62,-65,5.656854,3.417632e-06
23,"(4, 2)",-58,-64,-53,-61,-63,-65,6.082763,3.476267e-06
24,"(4, 2)",-59,-65,-53,-62,-64,-66,6.164414,3.519294e-06


In [ ]:
display(df.nsmallest(20,'euclidean_distance_mw', 'all'))

,gt_x_y,A1,A2,A3,A4,A5,A6,euclidean_distance,euclidean_distance_mw
48,"(2, 4)",-58,-61,-59,-64,-62,-65,2.828427,6.177862e-07
47,"(2, 4)",-58,-61,-59,-64,-61,-65,3.316625,6.713801e-07
53,"(0, 4)",-57,-61,-61,-66,-63,-66,4.123106,8.558678e-07
44,"(2, 0)",-57,-71,-60,-69,-71,-73,14.352700,9.013232e-07
54,"(0, 4)",-58,-62,-62,-67,-64,-68,5.656854,1.074193e-06
86,"(2, 2)",-59,-69,-61,-64,-69,-71,10.488088,1.215885e-06
40,"(2, 0)",-55,-69,-58,-67,-69,-71,10.488088,1.308645e-06
21,"(4, 4)",-61,-62,-57,-62,-61,-64,5.477226,1.337618e-06
15,"(6, 4)",-63,-64,-58,-62,-62,-65,6.557439,1.522556e-06
46,"(2, 4)",-61,-64,-62,-67,-65,-68,7.071068,1.563681e-06
